In [ ]:
%matplotlib inline 
from qiskit import QuantumCircuit, execute, Aer, IBMQ 
from qiskit.compiler import transpile, assemble 
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.tools.visualization  import circuit_drawer
from qiskit.extensions.standard import ccx
import numpy as np
from qiskit.providers.aer import AerError, QasmSimulator


circuit=0

def value(x, y, circuit):
    #2차 회로 만들기
    circuit = QuantumCircuit((len(x))*3+1,(len(x)+1))
    #X tofolli gate, cnot 적용
    for i in range(0, len(x)):
        if x[i] == "1":
            circuit.x(i)
    for j in range(0, len(y)):
        if y[j] == "1":
            circuit.x(len(x)+j)
    return circuit


def add(input1, input2, carryin, carryout, circuit):
    #CCNOT gate
    circuit.ccx(input1, input2, carryout)
    #CNOT gate
    circuit.cx(input1, input2)
    circuit.ccx(input2, carryin, carryout)
    circuit.cx(input2, carryin)
    circuit.cx(input1, input2)

x = '10101010'
y = '10001011'

circuit = value(x, y, circuit)


k=0
for i in reversed(range(len(x))):
    add(i, len(x)+i, len(x)+len(y)+k, len(x)+len(y)+k+1, circuit)
    k+=1

circuit.barrier()


for i in range(len(x)+1):
    circuit.measure([(len(x)*2)+i], [i])



#양자회로 시뮬레이션하기 위한 패킷
simulator = Aer.get_backend('qasm_simulator')
backend_opts_mps = {"method":"matrix_product_state"}
#result = execute(circuit, simulator, backend_options=backend_opts_mps).result()
result = execute(circuit, simulator).result()
key=list(result.get_counts().keys())[0]
print("x+y =",key)

plus=[]
num=[]
if len(key)>6:
    i=0
    for i in range(len(key)-9,len(key)-5):
        if key[i]=='1':
            key=key[:i]+'0'+key[i+1:len(key)]
            num.append(i)
    for i in range(len(key)-4):
        plus.append('0')
    for i in range(len(key)-9,len(key)-5):
        if i in num:
            plus.append('1')
        else:
            plus.append('0') 
    p=''.join(plus)

    k=0
    circuit=value(key,p,circuit)
    for i in reversed(range(len(key))):
        add(i,len(key)+i,len(key)+len(p)+k,len(key)+len(p)+k+1,circuit)
        k+=1
    circuit.barrier()
    for i in range(len(key)+1):
        circuit.measure([(len(key)*2)+i], [i])
    result = execute(circuit, simulator).result()
    key=list(result.get_counts().keys())[0]
    key=key[1:]
    
    one='000000001'
    if key[3]=='1':
        k=0
        circuit=value(key,one,circuit)
        for i in reversed(range(len(key))):
            add(i,len(key)+i,len(key)+len(one)+k,len(key)+len(one)+k+1,circuit)
            k+=1
        circuit.barrier()
        for i in range(len(key)+1):
            circuit.measure([(len(key)*2)+i], [i])
        simulator = Aer.get_backend('qasm_simulator')
        #backend_opts_mps = {"method":"matrix_product_state"}
        result = execute(circuit, simulator).result()
        key=list(result.get_counts().keys())[0]
        key=key[1:]
        key=key[:3]+'0'+key[4:]
key=key[len(key)-5:]

print("x+y(mod 31) =",key)
circuit.draw('mpl',scale=0.4)